<a href="https://colab.research.google.com/github/etuckerman/SOCOTEC/blob/main/FINAL_SOCOTEC_GPU_elliot_tuckerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `SOCOTEC` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `SOCOTEC`


In [2]:
from IPython.display import display, Markdown
import re
import ast
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define device (GPU if available, otherwise CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Function library
def add(a, b): return a + b
def square(a): return a ** 2
def cube(a): return a ** 3
def greet(name): return f"Hello, {name}!"
FUNCTION_MAPPING = {
    'add': add,
    'square': square,
    'cube': cube,
    'greet': greet
}


In [4]:
# # Prompt formatting function
# def format_prompt(input_text):
#     return f"""<|im_start|>system
# You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
# You convert natural language into the following functions:
# - add(a, b): Sum of a and b
# - square(a): a squared
# - cube(a): a cubed
# - greet(name): Returns a greeting for the name

# RULES:
# 1. Use ONLY these functions and their exact signatures: add, square, cube, greet.
# 2. If the input implies subtraction, division, modulo, or any other operation not listed above, return:
#    'SOCOTEC_ERROR_2'
# 3. Ensure correct parenthesis usage and proper nesting of these functions. Example:
#    Input: "Add the square of 3 to 10"
#    Output: add(square(3), 10)
# 4. If the input cannot be logically converted to one of the available functions, return:
#    'SOCOTEC_ERROR_1'
# 5. Do NOT generate functions that are not in this list: add, square, cube, greet.
# 6. Do NOT modify or invent function definitions.
# 7. If ambiguity exists in the input, assume a valid interpretation using only the allowed functions. If impossible, return:
#    'SOCOTEC_ERROR_1'

# IMPORTANT: You must ONLY return the function call, with no additional text, comments, or formatting.
# <|im_end|>

# <|im_start|>user
# {input_text}
# <|im_end|>

# <|im_start|>assistant
# """


In [5]:
# Model loading
model_id = "Qwen/Qwen2.5-Coder-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [6]:
# # Test cases
# test_cases = [
#     {
#         "input": "What is the sum of 10 and 5?",
#         "expected_call": "add(10, 5)"
#     },
#     {
#         "input": "What is the square of 6?",
#         "expected_call": "square(6)"
#     },
#     {
#         "input": "What is the cube of 3?",
#         "expected_call": "cube(3)"
#     },
#     {
#         "input": "What do you get when you add the square of 4 to 10?",
#         "expected_call": "add(square(4), 10)"
#     },
#     {
#         "input": "What do you get when you add the cube of 2 to 5?",
#         "expected_call": "add(cube(2), 5)"
#     },
#     {
#         "input": "What is the sum of the square of 4 and the cube of 3?",
#         "expected_call": "add(square(4), cube(3))"
#     },
#     {
#         "input": "Add the square of 5 to the cube of 2.",
#         "expected_call": "add(square(5), cube(2))"
#     },
#     {
#         "input": "Add the square of 4 to the cube of 3, then add that result to the square of 2.",
#         "expected_call": "add(add(square(4), cube(3)), square(2))"
#     },
#     {
#         "input": "Add the square of the result of adding 3 and 5 to the cube of 2.",
#         "expected_call": "add(square(add(3, 5)), cube(2))"
#     },
#     {
#         "input": "Add the square of the sum of 3 and 5 to the cube of 2, then add the result to the square of 4.",
#         "expected_call": "add(add(square(add(3, 5)), cube(2)), square(4))"
#     }
# ]

In [7]:
test_cases = [
    # Test case 1: Using `add`, `square`, and `greet` functions
    {
        "input": "Can you add the square of 4 to the result of greeting the name 'Alice'?",
        "expected_call": "add(square(4), greet('Alice'))"
    },

    # Test case 2: Using `cube` and `square` with a more complex arithmetic operation
    {
        "input": "What do you get when you cube 3, then add that to the square of 5?",
        "expected_call": "add(cube(3), square(5))"
    },

    # Test case 3: Using `add`, `square`, `cube`, and `greet`
    {
        "input": "First, square 6. Then, cube 2 and add that to the result of greeting 'Bob'. Finally, add the result to the square of 3.",
        "expected_call": "add(add(square(6), cube(2)), add(greet('Bob'), square(3)))"
    },

    # Test case 4: Introducing an undefined function `multiply`
    {
        "input": "Can you multiply 4 and 5, then add the result to the cube of 3?",
        "expected_call": "Error: Unsupported function: multiply"
    },

    # Test case 5: Trying to greet a name and perform arithmetic in the same call
    {
        "input": "Greet 'Charlie', then add the square of 7 to that result.",
        "expected_call": "add(greet('Charlie'), square(7))"
    },

    # Test case 6: Using a function `subtract` (not defined in the function mapping)
    {
        "input": "Subtract 10 from 20 and then cube 5.",
        "expected_call": "Error: Unsupported function: subtract"
    },

    # Test case 7: Complex combination of `add`, `square`, `cube`, and `greet`
    {
        "input": "First, square 9, then cube 2, and add those results. Afterward, greet 'David' and add that to the previous result.",
        "expected_call": "add(add(square(9), cube(2)), greet('David'))"
    },

    # Test case 8: Non-existent `divide` function to test error handling
    {
        "input": "Divide 20 by 5, and add that result to the square of 4.",
        "expected_call": "Error: Unsupported function: divide"
    },

    # Test case 9: Handling mixed functions (`add`, `cube`, and undefined `modulus`)
    {
        "input": "Cube 3, then add that to the result of modulus 10 by 3.",
        "expected_call": "Error: Unsupported function: modulus"
    },

    # Test case 10: Trying an undefined function `average`
    {
        "input": "What is the average of 2 and 4, then add that to the square of 3?",
        "expected_call": "Error: Unsupported function: average"
    }
]


In [8]:
# def generate_response(model, tokenizer, input_text):
#     """
#     Generate the assistant's response, extracting only the content between
#     <|im_start|>assistant and <|im_end|>.

#     Args:
#         model: The model object (e.g., Qwen or another compatible model).
#         tokenizer: The tokenizer object for the model.
#         input_text: The natural language input question.

#     Returns:
#         str: The assistant's response, between <|im_start|>assistant and <|im_end|>.
#     """
#     # Format the input prompt
#     prompt = format_prompt(input_text)  # Assuming format_prompt is defined earlier

#     # Tokenize the input and move to the model's device
#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

#     # Generate output using the model
#     outputs = model.generate(**inputs, max_new_tokens=250, num_return_sequences=1, do_sample=False, temperature=0.1)

#     # Decode the generated text
#     generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     # Split the generated text into lines
#     lines = generated_text.splitlines()

#     # Return the last line, stripped of extra whitespace
#     return lines[-1].strip() if lines else "SOCOTEC_ERROR_3"  # Fallback if empty

In [74]:
#TESTING WITH GENERATION_CONFIG.JSON SETUP:
def generate_response(model, tokenizer, input_text):
    """
    Generate the assistant's response, extracting only the content between
    <|im_start|>assistant and <|im_end|>.

    Args:
        model: The model object (e.g., Qwen or another compatible model).
        tokenizer: The tokenizer object for the model.
        input_text: The natural language input question.

    Returns:
        str: The assistant's response, between <|im_start|>assistant and <|im_end|> extracted.
    """
    # Format the input prompt
    prompt = format_prompt(input_text)  # Assuming format_prompt is defined earlier

    # Tokenize the input and move to the model's device
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Set generation parameters based on the provided config
    generation_params = {
        "max_new_tokens": 100,            # Maximum number of new tokens to generate
        "num_return_sequences": 1,       # Generate only one response
        "do_sample": True,               # Enable sampling (from config)
        "temperature": 0.01,              # Sampling temperature (from config)
        "top_p": 0.8,                    # Top-p (nucleus) sampling (from config)
        "top_k": 20,                     # Top-k sampling (from config)
        "repetition_penalty": 1.1        # Repetition penalty (from config)
    }

    # Generate output using the model with specified parameters
    outputs = model.generate(**inputs, **generation_params)

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Split the generated text into lines and return the last one
    lines = generated_text.splitlines()

    # Return the last line, stripped of extra whitespace
    return lines[-1].strip() if lines else "SOCOTEC_ERROR_3"  # Fallback if empty


In [10]:

def format_prompt(input_text):
    return f"""<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
You convert natural language into the following functions:
- add(a, b): Sum of a and b
- square(a): a squared
- cube(a): a cubed
- greet(name): Returns a greeting for the name

RULES:
1. Use ONLY these functions and their exact signatures: add, square, cube, greet.
2. If the input implies any operation not listed above, return:
   'ERROR: Unsupported functions: [list of unsupported functions]'
3. Ensure correct parenthesis usage and proper nesting of these functions. Example:
   Input: "Add the square of 3 to 10"
   Output: add(square(3), 10)
4. If the input cannot be logically converted to one of the available functions, return:
   'ERROR: Unable to convert input to valid functions. Explanation: [reason]'
5. Do NOT generate functions that are not in this list: add, square, cube, greet.
6. Do NOT modify or invent function definitions.
7. If ambiguity exists in the input, assume a valid interpretation using only the allowed functions. If impossible, return:
   'ERROR: Input is ambiguous or invalid. Explanation: [reason]'

IMPORTANT: You must ONLY return the function call or the error message, with no additional text, comments, or formatting.
<|im_end|>

<|im_start|>user
{input_text}
<|im_end|>

<|im_start|>assistant
"""

In [73]:
# Get user input
# user_question = input("Enter your question: ")
user_question = "6 square root 2 mod 5 div 10"
# Generate resposnse
generated_call = generate_response(model, tokenizer, user_question)
print("Generated Function Call:\n", generated_call, "\n")

Generated Function Call:
 system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
You convert natural language into the following functions:
- add(a, b): Sum of a and b
- square(a): a squared
- cube(a): a cubed
- greet(name): Returns a greeting for the name

RULES:
1. Use ONLY these functions and their exact signatures: add, square, cube, greet.
2. If the input implies any operation not listed above, return:
   'ERROR: Unsupported functions: [list of unsupported functions]'
3. Ensure correct parenthesis usage and proper nesting of these functions. Example:
   Input: "Add the square of 3 to 10"
   Output: add(square(3), 10)
4. If the input cannot be logically converted to one of the available functions, return:
   'ERROR: Unable to convert input to valid functions. Explanation: [reason]'
5. Do NOT generate functions that are not in this list: add, square, cube, greet.
6. Do NOT modify or invent function definitions.
7. If ambiguity exists in the input, assume a valid

In [12]:
# Get user input
# user_question = input("Enter your question: ")
user_question = "can we do subtract by using the add function with a negative number? like 5 - 2? "
# Generate resposnse
generated_call = generate_response(model, tokenizer, user_question)
print("Generated Function Call:\n", generated_call, "\n")

Generated Function Call:
 add(5, -2) 



In [13]:
# Get user input
# user_question = input("Enter your question: ")
user_question = "greet greet greet greet greet greet greet greet greet greet greet greet greet greet greet greet greet greet "
# Generate resposnse
generated_call = generate_response(model, tokenizer, user_question)
print("Generated Function Call:\n", generated_call, "\n")

Generated Function Call:
 ERROR: Unable to convert input to valid functions. Explanation: The input contains multiple instances of the same function without any logical operation or context. 



In [14]:
# Get user input
# user_question = input("Enter your question: ")
user_question = "greet paul as many times as the sun has rotated around the sun "
# Generate resposnse
generated_call = generate_response(model, tokenizer, user_question)
print("Generated Function Call:\n", generated_call, "\n")

Generated Function Call:
 'ERROR: Unable to convert input to valid functions. Explanation: The input involves an infinite loop (the sun rotating around itself) which cannot be represented with the given functions.' 



In [21]:
# Complex Edge Case Inputs
test_inputs = [
    "Add the square of the sum of 3 and 4 to the cube of 2",
    "What is the square of 6 divided by the cube of 2",
    "Add the square of 3 to the square of the sum of 2 and 5",
    "Add the square of the sum of 3 and 4, then add cube 2",
    "What is the cube of 3, the square of 2 plus 5 mod 10?",
    "Greet John and add the square of 4 to the cube of 2",
    "What is the sum of square and cube of 3?",
    "Add the square of the sum of 2 and 5 to greet 'John'",
    "Add the square of 3 to 5 mod 7",
    "Add the square of 4 and the cube of 2, then add the square of 3 to 10"
]

# Run the tests
for test_input in test_inputs:
    response = generate_response(model, tokenizer, test_input)
    print(f"Input: {test_input}\nResponse: {response}\n")


Input: Add the square of the sum of 3 and 4 to the cube of 2
Response: add(square(add(3, 4)), cube(2))

Input: What is the square of 6 divided by the cube of 2
Response: 'ERROR: Unsupported functions: [divide]'

Input: Add the square of 3 to the square of the sum of 2 and 5
Response: add(square(3), square(add(2, 5)))

Input: Add the square of the sum of 3 and 4, then add cube 2
Response: add(square(add(3, 4)), cube(2))

Input: What is the cube of 3, the square of 2 plus 5 mod 10?
Response: 'ERROR: Unsupported functions: [mod]'

Input: Greet John and add the square of 4 to the cube of 2
Response: greet('John'), add(square(4), cube(2))

Input: What is the sum of square and cube of 3?
Response: add(square(3), cube(3))

Input: Add the square of the sum of 2 and 5 to greet 'John'
Response: add(square(add(2, 5)), greet('John'))

Input: Add the square of 3 to 5 mod 7
Response: 'ERROR: Unsupported functions: [mod]'

Input: Add the square of 4 and the cube of 2, then add the square of 3 to 10
R

In [16]:
# 1. Unsupported Functions Error

# Test Case 1: Division (Unsupported Function)
test_case_1 = {
    "input": "What is 6 divided by the cube of 2?",
    "expected_response": "ERROR: Unsupported functions: [div]"
}

# Test Case 2: Modulo (Unsupported Function)
test_case_2 = {
    "input": "Add the square of 5 to 10 mod 3",
    "expected_response": "ERROR: Unsupported functions: [mod]"
}

# Test Case 3: Multiplication (Unsupported Function)
test_case_3 = {
    "input": "Multiply the square of 3 by 2",
    "expected_response": "ERROR: Unsupported functions: [multiply]"
}

# Test Case 4: Square Root (Unsupported Function)
test_case_4 = {
    "input": "What is the sqrt of 16?",
    "expected_response": "ERROR: Unsupported functions: [sqrt]"
}


# 2. Ambiguous or Incomplete Input Error

# Test Case 1: Incomplete Expression (Missing Operand)
test_case_5 = {
    "input": "Add square of 3 and",
    "expected_response": "ERROR: Unable to convert input to valid functions. Reason: Incomplete function structure."
}

# Test Case 2: Missing Operand After Function
test_case_6 = {
    "input": "What is the square of 3 plus",
    "expected_response": "ERROR: Unable to convert input to valid functions. Reason: Missing operand after function."
}


# 3. Unrecognized Function Error

# Test Case 1: Undefined Function (multiply)
test_case_7 = {
    "input": "Multiply the square of 3 by 2",
    "expected_response": "ERROR: Unsupported functions: [multiply]"
}

# Test Case 2: Undefined Function (sqrt)
test_case_8 = {
    "input": "What is the sqrt of 16?",
    "expected_response": "ERROR: Unsupported functions: [sqrt]"
}


# 4. Unclosed Parentheses Error

# Test Case 1: Unclosed Parenthesis in Expression
test_case_9 = {
    "input": "Add the square of 3 and (cube of 2",
    "expected_response": "ERROR: Unable to convert input to valid functions. Reason: Unclosed parentheses."
}


# 5. Multiple Function Error (Invalid Combinations)

# Test Case 1: Arithmetic and Greet Mixed Together
test_case_10 = {
    "input": "Add the square of 3 to greet John",
    "expected_response": "ERROR: Unable to convert input to valid functions. Reason: Invalid function combination (arithmetic and greet)."
}


# Running the tests
test_cases = [test_case_1, test_case_2, test_case_3, test_case_4, test_case_5, test_case_6,
              test_case_7, test_case_8, test_case_9, test_case_10]

for test_case in test_cases:
    response = generate_response(model, tokenizer, test_case["input"])
    print(f"Input: {test_case['input']}\nResponse: {response}\n")

Input: What is 6 divided by the cube of 2?
Response: 'ERROR: Unable to convert input to valid functions. Explanation: Division is not supported.'

Input: Add the square of 5 to 10 mod 3
Response: 'ERROR: Unsupported functions: [mod]'

Input: Multiply the square of 3 by 2
Response: 'ERROR: Unsupported functions: [multiply]'

Input: What is the sqrt of 16?
Response: ERROR: Unsupported functions: ['sqrt']

Input: Add square of 3 and
Response: add(square(3), 0)

Input: What is the square of 3 plus
Response: add(square(3), 0)

Input: Multiply the square of 3 by 2
Response: 'ERROR: Unsupported functions: [multiply]'

Input: What is the sqrt of 16?
Response: ERROR: Unsupported functions: ['sqrt']

Input: Add the square of 3 and (cube of 2
Response: add(square(3), cube(2))

Input: Add the square of 3 to greet John
Response: 'ERROR: Unsupported functions: [greet]'



In [17]:
slang_tests =[
"Yo, what's the square of 4 tho?",
"Aight, add the square of 3 to the cube of 2 fam.",
"Can you square the sum of 5 n 6 tho?",
"Yo, cube the number 3 for me real quick!",
"Bruh, what's the deal with adding 5 and 10?",
"Gimme the sum of 3 n 4, then square that!",
"Ayyo, can you greet my homie, John?"
]

for test in slang_tests:
  generated_call = generate_response(model, tokenizer, test)
  print(test)
  print(generated_call, "\n")

Yo, what's the square of 4 tho?
square(4) 

Aight, add the square of 3 to the cube of 2 fam.
add(square(3), cube(2)) 

Can you square the sum of 5 n 6 tho?
square(add(5, 6)) 

Yo, cube the number 3 for me real quick!
cube(3) 

Bruh, what's the deal with adding 5 and 10?
add(5, 10) 

Gimme the sum of 3 n 4, then square that!
square(add(3, 4)) 

Ayyo, can you greet my homie, John?
greet('John') 



In [18]:
long_test = "Add the square of the sum of 5 and 6 to the cube of 3, then add the square of the sum of 7 and 8, followed by adding the result to the cube of 2, then add the square of 9 to 10."
print(generate_response(model, tokenizer, long_test))

add(add(cube(3), square(add(5, 6))), add(add(cube(2), square(add(7, 8))), add(square(9), 10)))


In [19]:
res = add(add(cube(3), square(add(5, 6))), add(add(cube(2), square(add(7, 8))), add(square(9), 10)))


res

472

In [62]:
# streaming output / token by token output

from transformers import TextIteratorStreamer

def stream_response(model, tokenizer, input_text):
    """Streams the model's response token by token."""
    prompt = format_prompt(input_text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    streamer = TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        inputs,
        streamer=streamer,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.01,
        top_p=0.8,
        top_k=20,
        repetition_penalty=1.1
    )
    t = model.generate(**generate_kwargs)

    for new_text in streamer:
      print(new_text, end="", flush=True) # Print each token as it's generated
    print("\n")


In [71]:

# Example usage (replace with your actual input)
user_question = "Add the square of the sum of 5 and 6 to the cube of 3, then add the square of the sum of 7 and 8, followed by adding the result to the cube of 2, then add the square of 9 to 10."

stream_response(model, tokenizer, user_question)

add(add(cube(3), square(add(5, 6))), add(add(cube(2), square(add(7, 8))), add(square(9), 10)))

